In [ ]:
pip install kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import drive
import json
# json 파일 경로
file_path_train = '/content/drive/MyDrive/aikuthon/AIKUthon/train.json'
file_path_test = '/content/drive/MyDrive/aikuthon/AIKUthon/test.json'

# json 파일 열기 및 파싱
trainset = pd.read_json(file_path_train)
testset = pd.read_json(file_path_test)


In [ ]:
trainset[0]

{'info': {'caseField': '1',
  'detailField': '1',
  'trailField': '2',
  'caseNm': '대여금',
  'courtNm': '울산지방법원',
  'judmnAdjuDe': '2021. 05. 25.',
  'relateLaword': [],
  'qotatPrcdnt': []},
 'concerned': {'acusr': '1', 'dedat': '1'},
 'org': {'orgJdgmnCourtNm': '울산지방법원',
  'orgJdgmnAdjuDe': '2019. 7. 11. ',
  'orgJdgmnCaseNo': '2018가단68430'},
 'assrs': {'acusrAssrs': ['피고는 원고들에게 000에 투자하면 한 달에 400만 원의 수익을 얻을 수 있고, 4개월이지나면 원금도 반환받을 수 있다고 설명하면서 원고들을 현혹하여 투자금을 유치하였다.',
   '000는 전형적인 금융피라미드 수법의사기업체로서 원고들이 피고의 설명과 같은 수익을 올리거나 투자금을 반환받는 것이 불가능하였다.',
   '피고는 이를 알면서도 위와 같이 원고들을 기망하여 000에 투자하도록 하였거나, 설령 그렇지 않더라도 원고들에게 000에 대한 투자의 위험성을 제대로 알리지 않은 과실이 있다. ',
   '따라서 피고는 고의 또는 과실에 의한 불법행위에 대한 손해배상으로 원고들이 입은 위 투자금 상당의 손해를 배상할 책임이 있다.'],
  'dedatAssrs': []},
 'facts': ['원고 A은 2017. 12. 31. 0000에 거주하는 친구 F을 만나러 갔다가 당시 F의 집에 있던 F의 동생인 피고를 만나 비트코인 투자업체인 000에 대한 설명을 듣고 투자를 결심하였다.',
  '나머지 원고들은 2018. 1. 6.경 0000소재 피고의 여동생이 운영하는 식당으로 피고를 만나러 가서 피고로부터 000에 대한 설명을 듣고 각 투자를 결심하였다.',
  "원고 A은 2017. 12. 31. 1

# 데이터 전처리

In [9]:
def is_non_numeric(value):
    # 숫자 1, 2, 3, 4를 제외한 값이 있는지 확인
    return value not in ['1', '2', '3', '4', '5']
for concern in trainset['concerned']:
  if is_non_numeric(concern['dedat']):
    concern['dedat'] = 5
  if is_non_numeric(concern['acusr']):
    concern['acusr'] = 5


trainset['concerned'].head()

,concerned
0,"{'acusr': '1', 'dedat': '1'}"
1,"{'acusr': '1', 'dedat': 5}"
2,"{'acusr': '1', 'dedat': 5}"
3,"{'acusr': '1', 'dedat': '2'}"
4,"{'acusr': '1', 'dedat': '2'}"


In [22]:
trainset['assrs'][0]['acusrAssrs']
#trainset['facts'][0]
for i in trainset['facts']:
  print(type(i))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [ ]:
pip install transformers


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader

class LegalDataset(Dataset):
    def __init__(self, datasets, categories, labels, tokenizer, max_len):
        self.datasets = datasets
        self.categories = categories
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.datasets)

    def __getitem__(self, index):
        item = self.datasets[index]

        assrs = item['info.acusrAssrs'] + item['info.dedatAssrs'] # 문자열 리스트
        facts = item['info.facts'] # 문자열 리스트

        text = assrs + facts
        category = self.categories[index]
        label = self.labels[index]

        # 텍스트를 토큰화
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_txt = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        category_tensor = torch.tensor(category, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        return {
            'input_txt': input_txt,
            'attention_mask': attention_mask,
            'category': category_tensor,
            'label': label_tensor
        }


In [10]:
from transformers import BertTokenizer, BertModel
max_len =
batch_size =


# BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

train_dataset = LegalDataset(datasets=, categories=, labels=, tokenizer, max_len)
text_dataset = LegalDataset(datasets=, categories=, labels=, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
text_dataloader = DataLoader(text_dataset, batch_size = 64, shuffle = False)

model = BertModel.from_pretrained('monologg/kobert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

{'input_ids': tensor([[2, 0, 3]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4206, -0.2654,  0.0667,  ...,  0.0047, -0.2120, -0.1067],
         [ 0.1466,  0.2745,  0.1218,  ...,  0.1801, -0.2975, -0.1744],
         [ 0.0581, -0.2740,  0.2034,  ..., -0.4992, -0.4917,  0.2777]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 7.3357e-04, -6.6606e-02, -5.7220e-01,  7.5086e-02,  4.0251e-02,
          6.5434e-01, -6.5247e-01,  1.0613e-01,  8.8295e-02,  1.5081e-02,
          1.4897e-01, -7.3923e-02,  1.7384e-02,  1.9254e-01,  5.8342e-02,
         -3.1721e-02, -6.4720e-01, -8.6582e-02, -4.9349e-02, -8.4650e-02,
          6.9008e-02,  5.6884e-02, -6.3292e-02, -1.3818e-01, -5.2927e-02,
          5.6137e-02, -3.9697e-01,  5.5992e-03,  5.5259e-01, -5.8984e-02,
          6.4559e-01, -4.4364e-01, -6.6177e-02,  3.9578e-02, -8.5897e-01,
         -3.3367e-01, -9.5797e-03

In [ ]:


# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# 데이터셋 및 데이터로더 초기화
train_dataset = LegalDataset(file_path='trainset', tokenizer=tokenizer, max_len=max_len)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 데이터로더에서 배치 가져오기 예시
for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    print(input_ids)
    print(attention_mask)
    print(labels)
    break


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class LegalPredictionModel(nn.Module):
    def __init__(self, bert_model_name='monologg/kobert', num_categories=10, hidden_size=64):
        super(LegalPredictionModel, self).__init__()

        # BERT 모델 로드
        self.bert = BertModel.from_pretrained(bert_model_name)

        # 범주형 데이터 처리용 MLP
        self.category_dense = nn.Linear(num_categories, hidden_size)
        self.relu = nn.ReLU()

        # 결합된 출력 처리용 레이어
        bert_hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(bert_hidden_size + hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, category_input):
        # BERT 처리
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output

        # 범주형 데이터 처리
        category_output = self.relu(self.category_dense(category_input))

        # BERT 출력과 범주형 데이터 출력 결합
        combined_output = torch.cat((pooled_output, category_output), dim=1)

        # 최종 예측
        logits = self.classifier(combined_output)
        output = self.sigmoid(logits)

        return output

# 하이퍼파라미터 설정
max_len = 128
num_categories = 10  # 범주형 데이터의 차원

# 모델 초기화
model = LegalPredictionModel(num_categories=num_categories)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# 모델 요약
print(model)